# Hurdle 2 Code Walkthrough

## Introduction
All of the code discussed below resides under phase1-hurdles/hurdle2, available at:

https://github.com/SpectrumCollaborationChallenge/phase1-hurdles

Performer solutions for Hurdle 2 must be able to read complex samples from a TCP port, detect the locations of multiple instances of three different types of modulations and return the detected modulation type by bin number [0-30) using a Thrift RPC call. 

The Hurdle 2 folder includes code that runs inside the Test Infrastructure container. It also includes an example GNU Radio script to read the samples from the Test Infrastructure, and save them to disk. It also includes an example of how to modify autogenerated Python code from gnuradio-companion to add a Thrift RPC call one and only one time at the conclusion of a GNU Radio flowgraph. 

## The Build Script
build_hurdle_2_trusty.sh is a bash script that downloads a Ubuntu 14.04.5 64 bit LXC template and preconfigures it for use on Hurdle 2. It creates two containers, the Test Infrastructure container and the Solution container. The Solution container is a simple clone of the Test Infrastructure container. These containers will be stored in phase1-hurdles/hurdle2/lxc. Each container requires several GB of disk space, so ensure there is sufficient hard disk space before proceeding.


## Example Solution Code

### hurdle2.thrift
hurdle2.thrift is the Thrift IDL describing the RPC interface to be used for Hurdle 2. Participants should not modify this file other than to add name spaces for languages other than Python or C++. Participants choosing to implement their solution in language other than Python may use this IDL to autogenerate the Thrift interface code for their language of choice. See here for more information on generating source code in various languages using the Thrift compiler:

https://thrift.apache.org/tutorial

### hurdle2_solution_stub.grc and hurdle2_solution_stub_autogenerated.py
hurdle_2_solution_stub.grc is a GNU Radio flowgraph that runs in the Solution container that creates a TCP client to listen to samples generated by the Test Infrastructure container. hurdle2_solution_stub_autogenerated.py is the code generated by gnuradio-companion from hurdle2_solution_stub.grc. It is not an actual solution and should be replaced by Open Track teams.

Open track teams may use this file to record samples from the Test Infrastructure for offline processing during early development. The samples will be recorded in the standard GNU Radio Complex format. Users of MATLAB or Octave may process these files using utility scripts such as read_complex_binary.m. Teams may find this script in the GNU Radio source code under gnuradio/gr-utils/octave/read_complex_binary.m or at:

http://gnuradio.org/redmine/projects/gnuradio/repository/revisions/master/entry/gr-utils/octave/read_complex_binary.m

Python users may read these files with scipy as

In [1]:
import scipy

def read_samples_scipy(filename, blocklen):
    
    iq_samples = scipy.fromfile(filename, 
                                dtype=scipy.complex64, 
                                count=blocklen)
    return iq_samples

or with numpy as 

In [2]:
import numpy

def read_samples_numpy(filename, blocklen):
    
    iq_samples = numpy.fromfile(filename,
                                dtype=numpy.complex64,
                                count=blocklen)
    
    return iq_samples

### hurdle2_solution_stub_edited.py
hurdle_2_solution_stub_edited.py is a file based on hurdle2_solution_stub_autogenerated.py but including a few tweaks to manually add the ability to call a Thrift RPC function just before the flowgraph shuts down.

First the imports section is updated from:

In [3]:
from gnuradio import blocks
from gnuradio import eng_notation
from gnuradio import gr
from gnuradio.eng_option import eng_option
from gnuradio.filter import firdes
from optparse import OptionParser

to:

In [4]:
from gnuradio import blocks
from gnuradio import eng_notation
from gnuradio import gr
from gnuradio.eng_option import eng_option
from gnuradio.filter import firdes
from optparse import OptionParser

# ADDED BY HAND
import result_submitter

This imports the result_submitted module, which includes some utility functions for handling Thrift.

Next the **\_\_init\_\_()** statement of hurdle2_solution_stub_autogenerated is updated from:

In [5]:
class hurdle2_solution_stub_autogenerated(gr.top_block):

    def __init__(self, hurdle2_IQ_port='9091', hurdle2_result_port=9090, hurdle2_scoring_host='127.0.0.1', num_bins=30, sample_filename='sample_filename.dat'):
        gr.top_block.__init__(self, "Hurdle2 Solution Stub Autogenerated")

        ##################################################
        # Parameters
        ##################################################
        self.hurdle2_IQ_port = hurdle2_IQ_port
        self.hurdle2_result_port = hurdle2_result_port
        self.hurdle2_scoring_host = hurdle2_scoring_host
        self.num_bins = num_bins
        self.sample_filename = sample_filename

        ##################################################
        # Variables
        ##################################################
        self.samp_rate = samp_rate = 3e6

to:

In [6]:
class hurdle2_solution_stub_autogenerated(gr.top_block):

    def __init__(self, hurdle2_IQ_port='9091', hurdle2_result_port=9090, hurdle2_scoring_host='127.0.0.1', num_bins=30, sample_filename='sample_filename.dat'):
        gr.top_block.__init__(self, "Hurdle2 Solution Stub Autogenerated")

        ##################################################
        # Parameters
        ##################################################
        self.hurdle2_IQ_port = hurdle2_IQ_port
        self.hurdle2_result_port = hurdle2_result_port
        self.hurdle2_scoring_host = hurdle2_scoring_host
        self.num_bins = num_bins
        self.sample_filename = sample_filename

        ##################################################
        # Variables
        ##################################################
        self.samp_rate = samp_rate = 3e6
        
        # ADDED BY HAND
        self.answer = result_submitter.make_random_guess(num_bins)


This adds the **answer** member variable, so the final step can show an example of pulling a top level python variable out of the flowgraph. 

Finally, the **main()** function is updated from:

In [7]:
def main(top_block_cls=hurdle2_solution_stub_autogenerated, options=None):
    if options is None:
        options, _ = argument_parser().parse_args()

    tb = top_block_cls(hurdle2_IQ_port=options.hurdle2_IQ_port, hurdle2_result_port=options.hurdle2_result_port, hurdle2_scoring_host=options.hurdle2_scoring_host, num_bins=options.num_bins, sample_filename=options.sample_filename)
    tb.start()
    tb.wait()



to:

In [8]:
def main(top_block_cls=hurdle2_solution_stub_autogenerated, options=None):
    if options is None:
        options, _ = argument_parser().parse_args()

    tb = top_block_cls(hurdle2_IQ_port=options.hurdle2_IQ_port, hurdle2_result_port=options.hurdle2_result_port, hurdle2_scoring_host=options.hurdle2_scoring_host, num_bins=options.num_bins, sample_filename=options.sample_filename)
    tb.start()
    tb.wait()

    #ADDED BY HAND
    # If you put code after the final call to the top block's wait(), 
    # you can ensure that those functions will be called one and only one time
    
    # This will execute after you close the GNU Radio plots
    success = result_submitter.submit_my_answer(tb.answer, tb.hurdle2_scoring_host, tb.hurdle2_result_port)

    print("Results submitted successfully? {}".format(success))


which adds the Thrift RPC call at the end of the flowgraph. This RPC call should only be executed once at the end of the scenario.

### result_submitter.py
This module includes utility functions facilitate interacting with the Thrift server in the Test Infrastructure container. 

The first function, **submit_my_answer()**, initializes a Thrift client, opens a connection to the Thrift server, and submits an answer.

The success return value isn't in reference to the answer being right or wrong, it only means that the answer was properly formatted, ie the correct number and indecies of frequency bins. 

In [9]:
import random

from thrift import Thrift
from thrift.protocol import TBinaryProtocol
from thrift.transport import TSocket
from thrift.transport import TTransport

from hurdle2_rpc import Hurdle2Scoring
from hurdle2_rpc.ttypes import BinContents as BC


def submit_my_answer(answer, host, port):

    success = False
    try:

        # Make socket
        transport = TSocket.TSocket(host, port)

        # Buffering is critical. Raw sockets are very slow
        transport = TTransport.TBufferedTransport(transport)

        # Wrap in a protocol
        protocol = TBinaryProtocol.TBinaryProtocol(transport)

        # Create a client to use the protocol encoder
        # client = Calculator.Client(protocol)
        client = Hurdle2Scoring.Client(protocol)

        # Connect!
        transport.open()

        # send answer to the scoring server
        success = client.submitAnswer(answer)

        # Close!
        transport.close()

    except Thrift.TException as tx:
        print('%s' % tx.message)

    return success

The second function constructs a random answer for the Test Infrastructure container, selection a modulation type or noise from the enumerated list of potential modulations for each frequency bin.

In [10]:
def make_random_guess(num_bins):

    choices = [BC.NOISE, BC.FM, BC.GMSK, BC.QPSK]
    answer = {}

    for i in range(num_bins):
        answer[i] = random.choice(choices)

    return answer




## Test Infrastructure Code

### hurdle2_rpc
All code in the hurdle2_rpc subdirectory was autogenerated by running: 

```console
thrift --gen py -out ./ hurdle2.thrift
```

### generate_band_plan.py
This Python module generates a band plan of non-overlapping, randomly spaced modulation types with random bandwidths used by Transmitter.py to generate the complex baseband samples used for this hurdle.

### hurdle2:Transmitter.py
This Python module uses the output of generate_band_plan.py and constructs a set of modulated outputs according to the parameters in the band plan. It then makes these samples available on a TCP server.

### hurdle2:ScoringServer.py 
This Python module opens a Thrift server waiting for RPC calls from the participant solution container. Once the participant solution submits an answer for scoring, the ScoringServer compares the submitted answer against ground truth and generates a score. 

### run_hurdle2.py
This Python script pulls all the Test Infrastructure code together to execute Hurdle 2. 


## Items To Be Updated
All updates will be made available on:

https://github.com/SpectrumCollaborationChallenge/phase1-hurdles

Open Track teams will also be notified of updates via email. 

1. The SC2 team will be integrating the files in the Test Infrastructure container to require fewer commands to initiate a test.
2. generate_band_plan.py: This file does not currently respect the guard band spacing between signals as described in the Hurdle Problem Statement.
3. The spectral shaping and amplitudes of the modulations involved in the test will be updated to be representative of the final test.